<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# [Scénarios d'Exploration Statistique](https://github.com/wikistat/Exploration)

# Exploration de données d'enquête en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>:  explication du seuil de revenu 

### Résumé
Analyse de données extraites d'un recensement aux USA de 1994. Préparation des données, exploration élémentaire et multidimensionnelle ([ACP](http://wikistat.fr/pdf/st-m-explo-acp.pdf), [AFCM](http://wikistat.fr/pdf/st-m-explo-afcm.pdf)), [classification non supervisée](http://wikistat.fr/pdf/st-m-explo-classif.pdf). L'objectif est de préparer l'[étape suivante](https://github.com/wikistat/Apprentissage/tree/master/Adult-Census) consistant à modéliser / prévoir le niveau de revenu.

## Introduction
Des données publiques disponibles sur le site [UCI repository](http://archive.ics.uci.edu/ml/) sont extraites de la base de données issue du recensement réalisé aux Etats Unis en 1994. Ces données son largement utilisées et font référence comme outil de *benchmark* pour comparer les performances de méthodes d’apprentissage ou modélisation statistique. L’objectif est alors de prévoir la variable binaire « revenu annuel » supérieur ou inférieur à 50k$. Le travail décrit dans ce document est un préalable, il propose une exploration des données pour en appréhender les grandes structures. Il ne s’agit pas encore de données massives mais 32.561 individus sont décrits par les 14 variables du tableau ci-dessous :

Num| Libellé |	Ensemble de valeurs
-|--|--|--
1|`Age`|	real
2|	`workClass`|	Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
3|	`fnlwgt`|	real
4|	`education`|	Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool
5|	`educNum`|	integer
6|	`mariStat`|	Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse
7|	`occup`|	Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces
8|	`relationship`|	Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
9|	`origEthn`|	White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
10|	`sex`|	Female, Male
11|	`capitalGain`|	real  
12|	`capitalLoss`|	real
13|	`hoursWeek`|	real
14|	`nativCountry`|	United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands
15|	`income`|		>50K, <=50K

Une première étape permet de vérifier, sélectionner, recoder les données, avant d’aborder des outils plus élaborés. Les questions suivent la chronologie de ce déroulement.

La deuxième étape cherchant un meilleur modèle de prévision du dépassement de seuil de revenu est détaillée dans un [scénario](https://github.com/wikistat/Apprentissage/tree/master/Adult-Census) de la [saison 3: Apprentissage](https://github.com/wikistat/Apprentissage).


**Répondre aux questions en s'aidant des résultats des exécutions**.

## 1 Préparation des données


In [212]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

In [213]:
#Lecture
listeVar=['Age','workClass', 'fnlwgt', 'education', 'educNum', 'mariStat', 'occup', 'relationship',
          'origEthn', 'sex', 'capitalGain', 'capitalLoss', 'hoursWeek', 'nativCountry', 'income']

path="https://www.math.univ-toulouse.fr/~besse/Wikistat/data/adultCensus.dat"
dataBase = pd.read_csv(path, sep=',', header=None, index_col=False, names=listeVar,dtype={"workClass":object,"education":object,"mariStat":object,"occup":object,"relationship":object,"origEthn":object,"sex":object,"nativCountry":object,"income":object})
dataBase.head()

,Age,workClass,fnlwgt,education,educNum,mariStat,occup,relationship,origEthn,sex,capitalGain,capitalLoss,hoursWeek,nativCountry,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [214]:
#dataBase["workClass"]=pd.Categorical(dataBase["workClass"],ordered=False)
#dataBase["education"]=pd.Categorical(dataBase["education"],ordered=False)
#dataBase["mariStat"]=pd.Categorical(dataBase["mariStat"],ordered=False)
#dataBase["occup"]=pd.Categorical(dataBase["occup"],ordered=False)
#dataBase["relationship"]=pd.Categorical(dataBase["relationship"],ordered=False)
#dataBase["origEthn"]=pd.Categorical(dataBase["origEthn"],ordered=False)
#dataBase["sex"]=pd.Categorical(dataBase["sex"],ordered=False)
#dataBase["nativCountry"]=pd.Categorical(dataBase["nativCountry"],ordered=False)
#dataBase["income"]=pd.Categorical(dataBase["income"],ordered=False)
dataBase.dtypes

Age              int64
workClass       object
fnlwgt           int64
education       object
educNum          int64
mariStat        object
occup           object
relationship    object
origEthn        object
sex             object
capitalGain      int64
capitalLoss      int64
hoursWeek        int64
nativCountry    object
income          object
dtype: object

In [215]:
#workClass 
cat_workClass={" federal-gov": "Federal-Govt",
               " local-gov": "Other-Govt",
               " state-gov": "Other-Govt",
               " private": "Private",
               " self-emp-inc": "Self-Employed",
               " self-emp-not-inc": "Self-Employed",
               " without-pay": "Not-Working",
               " never-worked": "Not-Working"}
dataBase["workClass"]=dataBase.workClass.map(str.lower).map(cat_workClass)

In [216]:
#education
cat_education={" 10th": "Dropout",
               " 11th": "Dropout",
               " 12th": "Dropout",
               " 1st-4th":"Dropout",
               " 5th-6th": "Dropout",
               " 7th-8th": "Dropout",
               " 9th": "Dropout",
               " assoc-acdm": "Associates",
               " assoc-voc": "Associates",
               " bachelors": "Bachelors",
               " doctorate": "Doctorate",
               " hs-grad": "HS-Graduate",
               " masters": "Masters",
               " preschool": "Dropout",
               " prof-school": "Prof-School",
               " some-college": "HS-Graduate"}
dataBase["education"]=dataBase.education.map(str.lower).map(cat_education)

In [217]:
#mariStat
cat_mariStat={" never-married" : "Never-Married",
              " married-af-spouse" : "Married",
              " married-civ-spouse" : "Married",
              " married-spouse-absent" : "Not-Married",
              " separated" : "Not-Married",
              " divorced" : "Not-Married",
              " widowed" : "Widowed"
             }
dataBase["mariStat"]=dataBase.mariStat.map(str.lower).map(cat_mariStat)

In [218]:
#occup
cat_occup={" adm-clerical": "Admin",
           " armed-forces": "Military",
           " craft-repair": "Blue-Collar",
           " exec-managerial": "White-Collar",
           " farming-fishing": "Blue-Collar",
           " handlers-cleaners": "Blue-Collar",
           " machine-op-inspct": "Blue-Collar",
           " other-service": "Service",
           " priv-house-serv": "Service",
           " prof-specialty": "Professional",
           " protective-serv": "Other-occups",
           " sales": "Sales",
           " tech-support": "Other-occups",
           " transport-moving": "Blue-Collar",
          }
dataBase["occup"]=dataBase.occup.map(str.lower).map(cat_occup)
dataBase["occup"]=dataBase["occup"].replace("Miltary",np.nan)

In [219]:
#origEthn
cat_origEthn={" white": "White",
              " black": "Black",
              " amer-indian-eskimo": "Amer-Indian",
              " asian-pac-islander": "Asian",
              " other": "Other"
             }
dataBase["origEthn"]=dataBase.origEthn.map(str.lower).map(cat_origEthn)

In [220]:
#nativCountry
cat_nativCountry={" cambodia": "SE-Asia",
                  " canada": "British-Commonwealth",
                  " china": "China",
                  " columbia": "South-America",
                  " cuba": "Other",
                  " dominican-republic": "Latin-America",
                  " ecuador": "South-America",
                  " el-salvador": "South-America",
                  " england": "British-Commonwealth",
                  " france": "Euro_1",
                  " germany": "Euro_1",
                  " greece": "Euro_2",
                  " guatemala": "Latin-America",
                  " haiti": "Latin-America",
                  " holand-netherlands": "Euro_1",
                  " honduras": "Latin-America",
                  " hong": "China",
                  " hungary": "Euro_2",
                  " india": "British-Commonwealth",
                  " iran": "Other",
                  " ireland": "British-Commonwealth",
                  " italy": "Euro_1",
                  " jamaica": "Latin-America",
                  " japan": "Other",
                  " laos": "SE-Asia",
                  " mexico": "Latin-America",
                  " nicaragua": "Latin-America",
                  " outlying-us(guam-usvi-etc)": "Latin-America",
                  " peru": "South-America",
                  " philippines": "SE-Asia",
                  " poland": "Euro_2",
                  " portugal": "Euro_2",
                  " puerto-rico": "Latin-America",
                  " scotland": "British-Commonwealth",
                  " south": "Euro_2",
                  " taiwan": "China",
                  " thailand": "SE-Asia",
                  " trinadad&tobago": "Latin-America",
                  " united-states": "United-States",
                  " vietnam": "SE-Asia",
                  " yugoslavia": "Euro_2"
                 }
dataBase["nativCountry"]=dataBase.nativCountry.map(str.lower).map(cat_nativCountry)

In [221]:
#LcapitalGain et LcapitalLoss
dataBase["LcapitalGain"]=np.log(1+dataBase["capitalGain"])
dataBase["LcapitalLoss"]=np.log(1+dataBase["capitalLoss"])

In [ ]:
#capitalGain et capitalLoss
import math
dataBase["capitalGain"]=pd.cut(dataBase.capitalGain,bins=[-math.inf,0,??????????,labels=["None", "cgLow", "cgHigh"])

In [ ]:
  data[["capitalGain"]] <- ordered(cut(data$capitalGain,c(-Inf, 0, 
            median(data[["capitalGain"]][data[["capitalGain"]] >0]), Inf)), labels = c("None", "cgLow", "cgHigh"))
  data[["capitalLoss"]] <- ordered(cut(data$capitalLoss,c(-Inf, 0, 
            median(data[["capitalLoss"]][data[["capitalLoss"]] >0]), Inf)), labels = c("None", "clLow", "clHigh"))

  is.na(data) = data=='?'
  is.na(data) = data==' ?'
  
  
  data$mariStat = factor(data$mariStat)
  data$education = factor(data$education)
  data$nativCountry = factor(data$nativCountry)
  data$workClass = factor(data$workClass)
  data$occup = factor(data$occup)
  data$origEthn = factor(data$origEthn)
  data$sex = factor(data$sex)
  data$relationship = factor(data$relationship)
  data$income = as.factor(ifelse(data$income==data$income[1],"incLow","incHigh"))
  
   data$ageQ = cut(data$age,breaks=quantile(data$age,
                   probs=seq(0,1,1/5)),labels=c("Ag1","Ag2","Ag3","Ag4","Ag5"))
  data$hoursWeekQ = cut(data$hoursWeek,breaks=c(0,39,41,100),labels=c("HW1","HW2","HW3"))
  data = na.omit(data)
#  sample = rbinom(dim(data)[1],1,.3)           #splitting the data into training set
#  trainset = data[sample==0,]
#  valset = data[sample==1,]    
  
#  return(list(train = trainset,val = valset)) 
  return(data)
}
dataBase = dataPrepAdultCensus()

In [222]:
dataBase

,Age,workClass,fnlwgt,education,educNum,mariStat,occup,relationship,origEthn,sex,capitalGain,capitalLoss,hoursWeek,nativCountry,income,LcapitalGain,LcapitalLoss
0,39,Other-Govt,77516,Bachelors,13,Never-Married,Admin,Not-in-family,White,Male,2174,0,40,United-States,<=50K,7.684784,0.000000
1,50,Self-Employed,83311,Bachelors,13,Married,White-Collar,Husband,White,Male,0,0,13,United-States,<=50K,0.000000,0.000000
2,38,Private,215646,HS-Graduate,9,Not-Married,Blue-Collar,Not-in-family,White,Male,0,0,40,United-States,<=50K,0.000000,0.000000
3,53,Private,234721,Dropout,7,Married,Blue-Collar,Husband,Black,Male,0,0,40,United-States,<=50K,0.000000,0.000000
4,28,Private,338409,Bachelors,13,Married,Professional,Wife,Black,Female,0,0,40,Other,<=50K,0.000000,0.000000
5,37,Private,284582,Masters,14,Married,White-Collar,Wife,White,Female,0,0,40,United-States,<=50K,0.000000,0.000000
6,49,Private,160187,Dropout,5,Not-Married,Service,Not-in-family,Black,Female,0,0,16,Latin-America,<=50K,0.000000,0.000000
7,52,Self-Employed,209642,HS-Graduate,9,Married,White-Collar,Husband,White,Male,0,0,45,United-States,>50K,0.000000,0.000000
8,31,Private,45781,Masters,14,Never-Married,Professional,Not-in-family,White,Female,14084,0,50,United-States,>50K,9.552866,0.000000
9,42,Private,159449,Bachelors,13,Married,White-Collar,Husband,White,Male,5178,0,40,United-States,>50K,8.552367,0.000000


## 2 Description
**Q** Quelle est la population étudiée ? Quel est l’échantillon ?

**Q** Commenter, justifier les différentes transformations opérées. Repérer les variables quantitatives, qualitatives. Certaines (`age`, `hoursWeek`) sont présentes sous les deux types. Beaucoup de modalités on déjà été regroupées, notamment celles de `nativCountry` (voir le programme) certaines variables sont rendues qualitatives (`capitalLoss` ou `Gain`).

La popoulation étudiée est la population recensée aux Etats-Unis en 1994.